In [1]:
import pandas as pd
import glob
import numpy as np
import re
# Import the tabulate library
from tabulate import tabulate
from uszipcode import SearchEngine
import concurrent.futures

In [2]:
# Load dataset
participants = pd.read_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Revised Dataset/2013-2017 Bike MS Participants.csv')

# print
pd.set_option('display.max_columns',None)
participants.tail()

,Security Category Name,Fiscal Year,Internal Event Name,Event Date,Participation Type Name,Team Name,Team Creation Date,Team Division,Team ID,Contact ID,Member ID,Participant Accept Email,Registration Date,Registration Active Status,Is Team Captain,Is Secondary Registration,Is Prior Participant,Emails Sent,Total of All Confirmed Gifts($),Total From Participant($),Total Not From Participant($),Number From Participant,Number Not From Participant,Participant Email Status,Participant Employer,Participant Occupation,Participant Connection to MS,Address - Participant State/Province,Address - Participant County,Address - Participant City,Address - Participant ZIP/Postal Code,Registration Type,Event ID,Participant Gender,Participant Goal($),Suggested Participant Goal($),Source Code Type,Source Code Text,Sub Source Code Text
448837,WIG Bike Events,2014,"Wisconsin, Pewaukee - Bike MS: TOYOTA Best Dam...",8/2/2014 7:34,Virtual Cyclist,Team Wendy,8/5/2013 14:01,Friend and Family,351752.0,11165199,80015867,True,2/8/2014 16:47,Active,False,False,NaN,0,337.35,0.0,337.35,0,1,Bad (Hard Bounce),Aurora St. Luke's Medical Center,NaN,NaN,WI,Milwaukee,West Allis,53227,Online,22517,Female,1075.0,1075.0,NaN,NaN,NaN
448838,WIG Bike Events,2014,"Wisconsin, Pewaukee - Bike MS: TOYOTA Best Dam...",8/2/2014 7:34,Virtual Cyclist,Kari's Cadets,8/5/2013 15:43,Friend and Family,351768.0,8564626,64287610,True,7/8/2014 9:08,Active,False,False,NaN,0,520.00,0.0,520.00,0,13,Bad (Hard Bounce),NaN,Student,Friend has MS,MN,Dakota,Rochester,55901,Online,22517,Female,1075.0,1075.0,NaN,NaN,NaN
448839,WIG Bike Events,2014,"Wisconsin, Pewaukee - Bike MS: TOYOTA Best Dam...",8/2/2014 7:34,Virtual Cyclist,The Flying Monkeys,8/17/2013 11:32,Friend and Family,352278.0,13525569,85003404,True,7/6/2014 12:51,Active,False,False,NaN,0,0.00,0.0,0.00,0,0,Good,Productivity Associates Inc,Telecommunications,I have MS,VA,Newport News - Independent City,Newport News,23601,Online,22517,Male,1075.0,1075.0,6,www.facebook.com,/
448840,WIG Bike Events,2014,"Wisconsin, Pewaukee - Bike MS: TOYOTA Best Dam...",8/2/2014 7:34,Virtual Cyclist,NaN,NaN,NaN,NaN,11653982,81054272,True,1/29/2014 13:12,Active,False,False,NaN,2,300.00,255.0,45.00,2,2,Good,NaN,NaN,I have MS,WI,Calumet,Appleton,54915,Online,22517,Female,300.0,1075.0,NaN,NaN,NaN
448841,WIG Bike Events,2014,"Wisconsin, Pewaukee - Bike MS: TOYOTA Best Dam...",8/2/2014 7:34,Virtual Cyclist,NaN,NaN,NaN,NaN,3019528,40070347,True,7/9/2014 20:51,Active,False,False,Yes,0,1683.00,1.0,1682.00,1,64,Bad (Hard Bounce),Kitcinger,NaN,Relative has MS,WI,Milwaukee,South Milwaukee,53172,Offline,22517,Male,300.0,1075.0,NaN,NaN,NaN


In [3]:
dictionary = []
# Iterate through columns

# total rows
total_rows = len(participants)

for col in participants.columns:
    data_type = participants[col].dtype
    length = None
    if data_type == object:
        length = participants[col].str.len().max()
    
    length_format = '{:,.0f}'.format(length) if length is not None else None
    
    num_nulls = participants[col].isnull().sum()
    num_nulls_format = '{:,}'.format(num_nulls)
    percentage_null = (num_nulls/total_rows) * 100
    percentage_null_format = '{:,.2f}%'.format(percentage_null)
    num_distinct = participants[col].nunique()
    num_distinct_format = '{:,}'.format(num_distinct)
    
    # Initialize min and max values
    min_value = None
    max_value = None
    
    # Check if column is numeric
    if np.issubdtype(data_type, np.number):
        min_value = participants[col].min()
        max_value = participants[col].max()
        
    # Calculate avergae for numeric columns
    average = participants[col].mean() if np.issubdtype(data_type, np.number) else None
        
    #Sample values
    sample_values = participants[col].dropna().sample(5).tolist()
    
    #Append all info to dictionary list
    dictionary.append([col, data_type, length_format, num_nulls_format, percentage_null_format, num_distinct_format, min_value, max_value, average, sample_values])
    
# Create a df for the dictionary
dictionary_df = pd.DataFrame(dictionary, columns=['Column', 'Data_Type', 'Max_Character_Length', 'Null_Value_Count', 'Null_Percentage','Distinct_Value_Count', 'Min_Value', 'Max_Value','Average','Sample_Values'])

# So that Sample_Values are not truncated
pd.set_option('display.max_colwidth', 300)

# sort = dictionary_df.sort_values(by='Null_Value_Count')
# sort

dictionary_df

,Column,Data_Type,Max_Character_Length,Null_Value_Count,Null_Percentage,Distinct_Value_Count,Min_Value,Max_Value,Average,Sample_Values
0,Security Category Name,object,15,0,0.00%,51,NaN,NaN,NaN,"[MAM Bike Events, TXH Bike Events, TXH Bike Events, PAE Bike Events, PAE Bike Events]"
1,Fiscal Year,int64,None,0,0.00%,5,2013.0,2017.0,2.014829e+03,"[2016, 2017, 2015, 2016, 2016]"
2,Internal Event Name,object,81,0,0.00%,528,NaN,NaN,NaN,"[DED Bike MS: Bike to the Bay 2015, Michigan, Brighton - Bike MS: Fall Breakaway 2013, WAS 2016 Bike MS: Deception Pass Classic, KSG 2017 Bike MS: Wichita, MAM 2016 Bike MS: Cape Cod Getaway]"
3,Event Date,object,16,55,0.01%,496,NaN,NaN,NaN,"[9/24/2016 9:27, 8/26/2017 14:46, 6/11/2017 17:49, 9/20/2014 13:42, 5/18/2013 10:04]"
4,Participation Type Name,object,87,0,0.00%,459,NaN,NaN,NaN,"[Cyclist, Cyclist, Cyclist, Cyclist: Allegheny College Accommodation, Cyclist]"
5,Team Name,object,84,"89,223",19.88%,"16,773",NaN,NaN,NaN,"[Team kbroox, Mack Cycle, Team Coti, Concur, Sonic Streamers]"
6,Team Creation Date,object,16,"89,222",19.88%,"31,767",NaN,NaN,NaN,"[6/27/2013 17:06, 6/24/2012 8:09, 10/9/2014 14:25, 9/26/2013 14:13, 10/3/2016 19:47]"
7,Team Division,object,58,"125,632",27.99%,29,NaN,NaN,NaN,"[Friends and Family, Corporation, Friends and Family, Family and Friends, Other]"
8,Team ID,float64,None,"89,222",19.88%,"33,928",266344.0,538680.0,4.005711e+05,"[452921.0, 308168.0, 500653.0, 403072.0, 399266.0]"
9,Contact ID,int64,None,0,0.00%,"228,577",1564108.0,16365639.0,1.037348e+07,"[11361626, 12228349, 2776209, 14046001, 7895572]"


In [4]:
participants.shape[0]

448842

# Participant Gender

In [5]:
# Participant Gender value
max_length_value = participants['Participant Gender'].astype(str).apply(len).idxmax()
max_length = participants['Participant Gender'].astype(str).apply(len).max()
value_with_max_length = participants.at[max_length_value, 'Participant Gender']
value_with_max_length

# Get unique values from the column
unique_values = participants['Participant Gender'].unique()
unique_values


array(['Male', 'Female', nan, 'F'], dtype=object)

In [6]:
#Participant Gender

# Replace 'nan' with 'Unknown' and 'F' with 'Female' in the 'Participant Gender' column
participants['Participant Gender'] = participants['Participant Gender'].replace({'nan': 'Unknown', 'F': 'Female'})

In [7]:
num_nulls = participants['Participant Gender'].isnull().sum()
num_nulls

4455

# Address  -  Participant State/Province

In [8]:
participants.columns

Index(['Security Category Name', 'Fiscal Year', 'Internal Event Name',
       'Event Date', 'Participation Type Name', 'Team Name',
       'Team Creation Date', 'Team Division', 'Team ID', 'Contact ID',
       'Member ID', 'Participant Accept Email', 'Registration Date',
       'Registration Active Status', 'Is Team Captain',
       'Is Secondary Registration', 'Is Prior Participant', 'Emails Sent',
       'Total of All Confirmed Gifts($)', 'Total From Participant($)',
       'Total Not From Participant($)', 'Number From Participant',
       'Number Not From Participant', 'Participant Email Status',
       'Participant Employer', 'Participant Occupation',
       'Participant Connection to MS',
       'Address  -  Participant State/Province',
       'Address  -  Participant County', 'Address  -  Participant City',
       'Address -  Participant ZIP/Postal Code', 'Registration Type',
       'Event ID', 'Participant Gender', 'Participant Goal($)',
       'Suggested Participant Goal($)', '

In [11]:
# Get unique values from the column
unique_values = participants['Address  -  Participant State/Province'].unique()
unique_values

array(['CO', 'KS', 'PA', 'VA', 'MD', 'SC', 'IN', 'NY', 'CA', 'TX', 'NC',
       'OH', 'AK', 'NH', 'MN', 'ND', 'GA', 'IL', 'NJ', 'KY', 'FL', 'CT',
       'AR', 'MS', 'AL', 'TN', 'DE', 'LA', 'WI', 'MA', 'AZ', 'MI', 'MO',
       'NV', 'UT', nan, 'IA', 'ID', 'OK', 'OR', 'NE', 'WA', 'WY', 'AB',
       'NM', 'SD', 'DC', 'MT', 'HI', 'VT', 'RI', 'ME', 'QC', 'WV', 'YT',
       'GU'], dtype=object)

In [10]:
# List of invalid states
invalid_states = {'AE', 'ON', 'AP', 'BC', 'SK', 'NS', 'PR', 'VI', 'Newport', 'Alicante', 'MB', 'Columbia Heights', 'Minneapolis', 'Ridgefield Park', 'NB', 'Morristown', 'NL', 'Midwest City', 'Ardmore', 'Katy', 'Austin', 'Edinburg', 'Dallas', 'Canyon Lake', 'Houston', 'west haven', 'Greendale'}

# Replace invalid states with 'Unknown' and corresponding city and county
participants.loc[participants['Address  -  Participant State/Province'].isin(invalid_states), ['Address  -  Participant State/Province']] = np.nan


In [12]:
# State imputation

# Initialize the SearchEngine
search = SearchEngine()
# Example function to impute missing state based on ZIP code
def impute_state_from_zip(row):
    if pd.isnull(row['Address  -  Participant State/Province']) and not pd.isnull(row['Address -  Participant ZIP/Postal Code']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Address -  Participant ZIP/Postal Code'])
        if zipcode_info:
            # If there's a result, return the state associated with the ZIP code
            return zipcode_info.state
    # Return the original state if it's not missing or if ZIP code info wasn't found
    return row['Address  -  Participant State/Province']

# Apply the function to impute missing 'Donor State' values
participants['Address  -  Participant State/Province'] = participants.apply(impute_state_from_zip, axis=1)

# Check the result
participants[['Address -  Participant ZIP/Postal Code', 'Address  -  Participant State/Province']].sample(5)

,Address - Participant ZIP/Postal Code,Address - Participant State/Province
116890,1824,MA
343708,78756,TX
340330,77079,TX
432093,98112,WA
268998,8243,NJ


In [13]:
num_nulls = participants['Address  -  Participant State/Province'].isnull().sum()
num_nulls

1166

In [14]:
max_state_length = participants['Address  -  Participant State/Province'].str.len().max()
max_length_states = participants[participants['Address  -  Participant State/Province'].str.len() == max_state_length]['Address  -  Participant State/Province'].unique()

print("Maximum number of characters in the 'state' column:", max_state_length)
print("State(s) with the maximum number of characters:", max_length_states)

Maximum number of characters in the 'state' column: 2.0
State(s) with the maximum number of characters: ['CO' 'KS' 'PA' 'VA' 'MD' 'SC' 'IN' 'NY' 'CA' 'TX' 'NC' 'OH' 'AK' 'NH'
 'MN' 'ND' 'GA' 'IL' 'NJ' 'KY' 'FL' 'CT' 'AR' 'MS' 'AL' 'TN' 'DE' 'LA'
 'WI' 'MA' 'AZ' 'MI' 'MO' 'NV' 'UT' 'IA' 'ID' 'OK' 'AE' 'OR' 'NE' 'WA'
 'WY' 'AB' 'NM' 'SD' 'DC' 'MT' 'AP' 'HI' 'VT' 'RI' 'ME' 'QC' 'WV' 'PR'
 'VI' 'YT' 'GU']


In [15]:
states_with_more_than_2_chars = participants[participants['Address  -  Participant State/Province'].str.len() > 2]['Address  -  Participant State/Province'].unique()
print("Unique values in 'state' column with more than 2 characters:", states_with_more_than_2_chars)

Unique values in 'state' column with more than 2 characters: []


# Address  -  Participant City

In [16]:
max_length_value = participants['Address  -  Participant City'].astype(str).apply(len).idxmax()
max_length = participants['Address  -  Participant City'].astype(str).apply(len).max()
value_with_max_length = participants.at[max_length_value, 'Address  -  Participant City']
value_with_max_length

'Tootgarook, Victoria, Australia'

In [17]:
num_nulls = participants['Address  -  Participant City'].isnull().sum()
num_nulls

137

In [18]:
# City imputation

# Initialize the SearchEngine
search = SearchEngine()
# Example function to impute missing state based on ZIP code
def impute_state_from_zip(row):
    if pd.isnull(row['Address  -  Participant City']) and not pd.isnull(row['Address -  Participant ZIP/Postal Code']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Address -  Participant ZIP/Postal Code'])
        if zipcode_info:
            # If there's a result, return the state associated with the ZIP code
            return zipcode_info.state
    # Return the original state if it's not missing or if ZIP code info wasn't found
    return row['Address  -  Participant City']

# Apply the function to impute missing 'Donor State' values
participants['Address  -  Participant City'] = participants.apply(impute_state_from_zip, axis=1)

# Check the result
participants[['Address -  Participant ZIP/Postal Code', 'Address  -  Participant City']].sample(5)

,Address - Participant ZIP/Postal Code,Address - Participant City
323133,77092,Houston
2599,35622,Falkville
96026,68955,Juniata
190995,27502,Apex
358814,53704,Madison


In [19]:
# Initialize the SearchEngine
search = SearchEngine()

# Adjusted function to handle NaN values
def check_and_collect_invalid_city(row):
    zip_code = row['Address -  Participant ZIP/Postal Code']
    city_name = row['Address  -  Participant City']
    
    # Check if city_name is NaN (or equivalently, a float) before proceeding
    if pd.isna(city_name):
        # Optionally, handle NaN city names differently, e.g., skip or mark as invalid
        return
    
    # Perform ZIP code lookup
    result = search.by_zipcode(zip_code)
    if result and result.city:
        # Use str() to ensure we're calling .lower() on a string object
        if result.city.lower() != str(city_name).lower():
            invalid_cities.append(city_name)

# List to hold invalid city names
invalid_cities = []

# Apply the function to each row in the DataFrame
participants.apply(check_and_collect_invalid_city, axis=1)

# Create a DataFrame from the list of invalid cities for easier analysis
invalid_cities_df = pd.DataFrame(invalid_cities, columns=['Invalid City'])
invalid_cities_df

,Invalid City
0,Anywhere
1,Thornton
2,City
3,Fairbanks
4,Fairbanks
...,...
39005,Whitefish Bay
39006,Wauwatosa
39007,Whitefish Bay
39008,Greenfield


In [20]:
# Initialize the SearchEngine
search = SearchEngine()

# Define a function to replace invalid city names with NaN directly in the DataFrame
def replace_invalid_city_with_nan(row):
    # Extract ZIP code and city from the row
    zip_code = row['Address -  Participant ZIP/Postal Code']
    city_name = row['Address  -  Participant City']
    
    # Perform the ZIP code lookup
    result = search.by_zipcode(zip_code)
    
    # Check if the result is valid and if the city name matches
    if result and result.city:
        if result.city.lower() != city_name.lower():
            # If the city name is invalid, return NaN
            return np.nan
    # Return the original city name if it's valid or if there's no result for the ZIP code
    return city_name

# Apply the function to the 'Address - Participant City' column and update it
participants['Address  -  Participant City'] = participants.apply(replace_invalid_city_with_nan, axis=1)

# Contact ID

In [21]:
# Contact ID
max_length_value = participants['Contact ID'].astype(str).apply(len).idxmax()
max_length = participants['Contact ID'].astype(str).apply(len).max()
value_with_max_length = participants.at[max_length_value, 'Contact ID']
value_with_max_length

12495541

In [22]:
# Convert 'Contact ID' to pandas nullable integer type
participants['Contact ID'] = pd.to_numeric(participants['Contact ID'], errors='coerce').astype('Int64')
participants['Contact ID']

0         12495541
1         12314585
2         15440936
3          5990607
4         14568407
            ...   
448837    11165199
448838     8564626
448839    13525569
448840    11653982
448841     3019528
Name: Contact ID, Length: 448842, dtype: Int64

# Participant Goal($)

In [23]:
# Convert 'Participant Goal($)' column to string
participants['Participant Goal($)'] = participants['Participant Goal($)'].astype(str)

# Now apply .str.contains() with the special character pattern
special_char_pattern = r'[^0-9.]'
special_char_rows = participants[participants['Participant Goal($)'].str.contains(special_char_pattern, regex=True, na=False)]

# Display the relevant columns for the rows with special characters
relevant_columns = ['Participant Goal($)']
special_char_data = special_char_rows[relevant_columns]

# Display the DataFrame with the values that contain special characters
special_char_data

,Participant Goal($)


In [24]:
# Define a custom function to convert to float
def convert_to_float(value):
    if value is not None:
        return float(value)
    return None

# Apply the custom function to 'Participant Goal($)' column
participants['Participant Goal($)'] = participants['Participant Goal($)'].apply(convert_to_float)
participants['Participant Goal($)']

0         1095.0
1         1095.0
2         1095.0
3         1095.0
4         1095.0
           ...  
448837    1075.0
448838    1075.0
448839    1075.0
448840     300.0
448841     300.0
Name: Participant Goal($), Length: 448842, dtype: float64

# Team ID

In [25]:
# Team (ID)
max_length_value = participants['Team ID'].astype(str).apply(len).idxmax()
max_length = participants['Team ID'].astype(str).apply(len).max()
value_with_max_length = participants.at[max_length_value, 'Team ID']
value_with_max_length

# filtered values 
values = participants[participants['Team ID'].astype(str).apply(len) > 5][['Team ID']]

values

,Team ID
2,451067.0
3,449427.0
9,452531.0
11,453915.0
13,452531.0
...,...
448835,352278.0
448836,395458.0
448837,351752.0
448838,351768.0


In [26]:
# Convert 'Team ID' to pandas nullable integer type
participants['Team ID'] = pd.to_numeric(participants['Team ID'], errors='coerce').astype('Int64')
participants['Team ID']

0           <NA>
1           <NA>
2         451067
3         449427
4           <NA>
           ...  
448837    351752
448838    351768
448839    352278
448840      <NA>
448841      <NA>
Name: Team ID, Length: 448842, dtype: Int64

# Event ID

In [27]:
# EventID dirty value
max_length_value = participants['Event ID'].astype(str).apply(len).idxmax()
max_length = participants['Event ID'].astype(str).apply(len).max()
value_with_max_length = participants.at[max_length_value, 'Event ID']
value_with_max_length

# filtered values 
values = participants[participants['Event ID'].astype(str).apply(len) > 4][['Event ID']]

values

,Event ID
0,26846
1,26886
2,26886
3,26886
4,26886
...,...
448837,22517
448838,22517
448839,22517
448840,22517


In [28]:
# Convert 'Team ID' to pandas nullable integer type
participants['Event ID'] = pd.to_numeric(participants['Event ID'], errors='coerce').astype('Int64')
participants['Event ID']

0         26846
1         26886
2         26886
3         26886
4         26886
          ...  
448837    22517
448838    22517
448839    22517
448840    22517
448841    22517
Name: Event ID, Length: 448842, dtype: Int64

In [29]:
participants.columns

Index(['Security Category Name', 'Fiscal Year', 'Internal Event Name',
       'Event Date', 'Participation Type Name', 'Team Name',
       'Team Creation Date', 'Team Division', 'Team ID', 'Contact ID',
       'Member ID', 'Participant Accept Email', 'Registration Date',
       'Registration Active Status', 'Is Team Captain',
       'Is Secondary Registration', 'Is Prior Participant', 'Emails Sent',
       'Total of All Confirmed Gifts($)', 'Total From Participant($)',
       'Total Not From Participant($)', 'Number From Participant',
       'Number Not From Participant', 'Participant Email Status',
       'Participant Employer', 'Participant Occupation',
       'Participant Connection to MS',
       'Address  -  Participant State/Province',
       'Address  -  Participant County', 'Address  -  Participant City',
       'Address -  Participant ZIP/Postal Code', 'Registration Type',
       'Event ID', 'Participant Gender', 'Participant Goal($)',
       'Suggested Participant Goal($)', '

# Primary Key

In [33]:
# drop column
#participants = participants.drop(columns=['participant_ID'])

# Drop unnecessary columns

In [30]:
columns_to_keep = ['Participant Gender', 'Address  -  Participant State/Province', 'Address  -  Participant City', 'Participant Goal($)', 'Contact ID', 'Team ID','Team Name','Event ID']

participants = participants[columns_to_keep]
participants.columns

Index(['Participant Gender', 'Address  -  Participant State/Province',
       'Address  -  Participant City', 'Participant Goal($)', 'Contact ID',
       'Team ID', 'Team Name', 'Event ID'],
      dtype='object')

# Rename columns

In [31]:
# Create a copy of the participants DataFrame
participants_copy = participants.copy()

# Rename the columns on the copy
participants_copy.rename(
    columns={
        "Participant Gender": "p_gender",
        "Address  -  Participant State/Province": "p_state",
        "Address  -  Participant City": "p_city",
        "Participant Goal($)": "p_goal_amt",
        "Contact ID": "contactID",
        "Team ID": "teamID",
        "Team Name": "t_name",
        "Event ID": "eventID"
    },
    inplace=True
)

# Check the columns of the copied DataFrame
participants_copy.columns

Index(['p_gender', 'p_state', 'p_city', 'p_goal_amt', 'contactID', 'teamID',
       't_name', 'eventID'],
      dtype='object')

# Tables

In [32]:
# participant
participant = participants_copy[['contactID','p_gender','p_state','p_city']].copy()
# Remove duplicates
participant_cleaned = participant.drop_duplicates(subset=['contactID'])

participant_cleaned.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/participant.csv', index=False)

In [33]:
# registration
registration = participants_copy[['contactID','eventID','teamID']].copy()
# Remove duplicates
registration_cleaned = registration.drop_duplicates()

registration_cleaned.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/registration.csv', index=False)

In [36]:
# team
team = participants_copy[['teamID','t_name']].copy()
# Remove duplicates
team_cleaned = team.drop_duplicates()

# Drop rows where both 'teamID' and 't_name' are null
team_cleaned.dropna(subset=['teamID', 't_name'], how='all', inplace=True)

team_cleaned.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/team.csv', index=False)


C:\Users\SANTIAGO\AppData\Local\Temp\ipykernel_17432\2509059030.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_cleaned.dropna(subset=['teamID', 't_name'], how='all', inplace=True)
